# Format datas

## Libs :

In [1]:
import json
import numpy as np
import pandas as pd
import os

## Parameters :

In [2]:
path = "output/romain/" #do not for the / at the end of the path
rssi_csv_output = "rssi.csv"

# preparing datas :
# for removing access points :
ap_ignoring_threshold = 0.1 # 0 - 1 ; minimum occurence average that an access point need to have in zones
#if more then the value -95 will be used, if less, the access point will be removed
#threshold (0 - 1) to average when missing RSSI :
acquisition_average_threshold = 0.8 # in an acquisition within a certain position
zone_average_threshold = 0.7 # in a zone between the positions

## Variables :

## Import datas :

In [3]:
file_path = 'output/romain/h106.json' #temporary for test


    
# list the mac in all zones to create a consistent index in all rssi array
def listMac(path):
    m_mac = []
    for zone_file in os.listdir(path):
        # Opening JSON file
        f = open(path + zone_file)

        # returns JSON object as a dictionary
        data = json.load(f)
        for position in data:
            i_acquisition = -1
            for acquisition in data[position]:
                i_acquisition += 1
                n_access_points = len(data[position][i_acquisition])
                for i_access_point in range(n_access_points):
                    i_mac = data[position][i_acquisition][i_access_point]['mac']
                    #check if mac address is already known :
                    if not i_mac in m_mac:
                        m_mac.append(i_mac) #if not, add it in the mac array
    return m_mac

mac = listMac(path)

def importZone(zone_file_path):
    m_position_changes = [0] # acquisition numbers corresponding to a change of position

    # Opening JSON file
    f = open(zone_file_path)

    # returns JSON object as a dictionary
    data = json.load(f)

    #print("number of positions : ",len(data))
    #print("number of acquisitions (first acquisition) : ",len(data['output4']))
    #print("number of RSSI detected (first acquisition) : ",len(data['output4'][0]))
    #print("mac and RSSI (first acquisition) : ",len(data['output4'][0][0]))

    #compute the number of acquisitions (and saving the position changes) :
    n_acquisition = 0
    i_position = -1
    for position in data:
        i_position += 1
        n_acquisition += len(data[position])
        m_position_changes.append(n_acquisition)

    #print(m_position_changes)
    n_mac = len(mac)
    #print("number of acquisition : ", n_acquisition)
    #print("number of access points (mac) detected : ", n_mac)

    #initialize the array with all the rssi values :
    m_rssi = np.zeros([n_acquisition, n_mac])

    #print(data['output4'][1])

    acquisition_index_for_rssi_array = -1
    i_position = -1
    for position in data:
        i_acquisition = -1
        i_position += 1
        #print("\n",i_position)
        for acquisition in data[position]:
            i_acquisition += 1
            acquisition_index_for_rssi_array += 1
            n_access_points = len(data[position][i_acquisition])
            #print("\n",i_acquisition)
            #print(n_access_points)
            #print(len(data[position][i_acquisition]))
            for i_access_point in range(n_access_points):
                #print("\n",i_access_point)
                i_mac = data[position][i_acquisition][i_access_point]['mac']
                #print("mac : ",m_mac)
                i_rssi = data[position][i_acquisition][i_access_point]['RSSI']
                #print("RSSI : ",m_rssi)
                #print(data[position][i_acquisition][i_access_point])
                #add the RSSI value in the rssi array :
                #print(m_mac.index(i_mac))
                m_rssi[acquisition_index_for_rssi_array, mac.index(i_mac)] = i_rssi

    return m_rssi, m_position_changes

rssi, position_changes = importZone(file_path)
print(position_changes)

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]


## prepare datas

In [4]:
def correctDataWithinPosition():
    # average within a position :
    # for each access point
    for access_point in range(len(mac)):
        #for each position :
        for position in range(len(position_changes) - 1) :
            number_of_zeros = 0
            number_of_acquisitions = position_changes[position + 1] - position_changes[position]
            #print(position_changes[position])
            # computing the number of 0 for the position
            for acquisition in range(number_of_acquisitions):
                #print(position_changes[position] + acquisition)
                if rssi[position_changes[position] + acquisition, access_point] == 0: #if rssi = 0 aka not detected
                    #print(position_changes[position] + acquisition, access_point) # print the indexs of the RSSI
                    number_of_zeros += 1
            #print(number_of_zeros)
            #print(number_of_acquisitions)
            if number_of_zeros/number_of_acquisitions <= 1.000000000000001 - acquisition_average_threshold:            
                #the number of zeros is low enough to fill the value of rssi with the average
                #computing the average :
                rssi_acquisition_values = rssi[position_changes[position]:position_changes[position + 1], access_point]
                rssi_acquisition_values = rssi_acquisition_values[rssi_acquisition_values != 0]
                average_value = np.average(rssi_acquisition_values)
                #print(rssi_acquisition_values)
                for acquisition in range(number_of_acquisitions):
                    if rssi[position_changes[position] + acquisition, access_point] == 0: #if rssi = 0 aka not detected
                        rssi[position_changes[position] + acquisition, access_point] = average_value
correctDataWithinPosition()

# average within zone :
def correctDataWithinZone():
    # for each access point
    number_of_acquisitions = position_changes[-1] # position_changes[-1] correspond to the number of acquisitions
    for access_point in range(len(mac)):
        number_of_zeros = 0
        for acquisition in range(number_of_acquisitions): 
            if rssi[acquisition, access_point] == 0: #if rssi = 0 aka not detected
                number_of_zeros += 1
        if number_of_zeros == 0:
            pass # nothing to do as each rssi value in the array is filled
        elif number_of_zeros/number_of_acquisitions <= 1.000000000000001 - zone_average_threshold:
            #the average will be used to complete
            #computing the average :
            rssi_acquisition_values = rssi[:, access_point]
            rssi_acquisition_values = rssi_acquisition_values[rssi_acquisition_values != 0]
            average_value = np.average(rssi_acquisition_values)
            for acquisition in range(number_of_acquisitions):
                if rssi[acquisition, access_point] == 0: #if rssi = 0 aka not detected
                    rssi[acquisition, access_point] = average_value

def correctDataByDeletingMac():
    m_access_point_to_delete = []
    number_of_acquisitions = position_changes[-1] # position_changes[-1] correspond to the number of acquisitions
    # for each access point
    for access_point in range(len(mac)):
        number_of_zeros = 0
        for acquisition in range(number_of_acquisitions): 
            if rssi[acquisition, access_point] == 0: #if rssi = 0 aka not detected
                number_of_zeros += 1
        if number_of_zeros == 0:
            pass # nothing to do as each rssi value in the array is filled       
        elif number_of_zeros/number_of_acquisitions <= 1.000000000000001 - ap_ignoring_threshold: # a deplacer pour toutes les zones
            #filling the RSSI value with -95 as it's not detected and we suppose the access point is too far
            for acquisition in range(number_of_acquisitions):
                if rssi[acquisition, access_point] == 0: #if rssi = 0 aka not detected
                    rssi[acquisition, access_point] = -95
        else:
            #delete the access point with not enough measures
            #print("this access point will be deleted : ", rssi[:, access_point])
            m_access_point_to_delete.append(access_point)
            #rssi = np.delete(rssi, access_point, 1)
    return m_access_point_to_delete

access_point_to_delete = correctDataByDeletingMac()
        
print(len(access_point_to_delete), "access point will be deleted")
print(np.shape(rssi))
rssi = np.delete(rssi, access_point_to_delete, 1)
print(np.shape(rssi))
                
# convert array into dataframe
DF = pd.DataFrame(rssi)
  
# save the dataframe as a csv file
DF.to_csv(rssi_csv_output)

166 access point will be deleted
(55, 270)
(55, 104)
